In [1]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Research essay A0284714Y.pdf")

documents = loader.load()


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

textSplitting = RecursiveCharacterTextSplitter(
    chunk_size = 800, chunk_overlap = 0, 
    length_function = len, is_separator_regex = False
)

chunks = textSplitting.split_documents(documents)

print(len(chunks))

25


In [3]:
from sentence_transformers import SentenceTransformer
import numpy


embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

chunk_texts = [chunk.page_content for chunk in chunks]

embeddings = embedding_model.encode(chunk_texts, show_progress_bar=True)

embeddings_np = numpy.array(embeddings).astype('float32') #fass takes in 



c:\UnityProjects\OrbitalKayo\KAYO\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\UnityProjects\OrbitalKayo\KAYO\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 1/1 [00:02<00:00,  2.68s/it]


In [4]:
import faiss #facebook ai similarity search 

query = "tourism in india"

query_embedding = embedding_model.encode(query, show_progress_bar=True)

query_embedding_np = numpy.array([query_embedding]).astype('float32')

nearest_neighbours = 5

dimension = embeddings_np.shape[1]

index = faiss.IndexFlatL2(dimension)
 
index.add(embeddings_np)

distance, indices = index.search(query_embedding_np, nearest_neighbours)

#The lower the distance, the more similar the chunk is to your query.
#hese are the positions of the most similar chunks within your original chunks list

print("Distance: ", distance)


print("Indices: ", indices)

similarity_scores = [ 1 / (1 + dist) for dist in distance[0] ]

print("Similarity scores: ", similarity_scores)


Batches: 100%|██████████| 1/1 [00:00<00:00, 62.04it/s]

Distance:  [[0.647101   0.6590144  0.6618579  0.67796177 0.718436  ]]
Indices:  [[ 3 13 20  2  4]]
Similarity scores:  [0.6071273158576378, 0.6027675213050624, 0.6017361642006633, 0.5959611356073153, 0.5819244932081126]


In [5]:
#Retrieving the chunks using distance and indices

for i in range(len(indices)) :
    for rank in range(nearest_neighbours):
        chunk_index = indices[i][rank]
        similarity = similarity_scores[rank]
        chunk_text = chunks[chunk_index]
        print(f"Rank {rank+1}: {chunk_text.page_content} with similarity {similarity}")
        

Rank 1: understanding further deepening bilateral ties.   This essay seeks to explore the contribution of Tourism in enhancing the ties between Singapore and India. Particularly, diving into the trends and strategies employed by both countries regarding tourism, along with their effects on various levels and further analysis of the challenges and the solutions beneficial for both nations.      Overview of Tourism Trends and Strategies   In the early years after Singapore's independence in the 1960s, the two nations began collaborating to promote tourism and cultural exchanges. Singapore leveraged its position as a safe, efficient, and business-friendly destination and India's "Incredible India" campaign launched in 2002 helped raise awareness of India's diverse tourism offerings. In recent years, with similarity 0.6071273158576378
Rank 2: campaigns aimed at increasing inbound travel to Singapore. With two offices in Mumbai and Delhi, STB promotes Singapore attractions as a holiday dest

In [6]:
#LLM 
import os
import google.generativeai as genai

api_key = os.getenv("GENAI_API_KEY")
if not api_key:
    raise ValueError("API key not found in environment variables.")

genai.configure(api_key=api_key)

llm = genai.GenerativeModel('gemini-1.5-flash')

response = llm.generate_content("What is life")

print(response.text)

ValueError: API key not found in environment variables.

In [20]:
import os
api_key = os.getenv("GENAI_API_KEY")
print(api_key)

None
